In [17]:
from importlib import reload
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style='whitegrid')
import ard_logging
import logging
ard_logging.init(force_debug=True)
LOG = logging.getLogger(__name__)
LOG.info('Hello from notebook')

df = sns.load_dataset('diamonds')
df.info()
df.head()

2022-06-20 13:45:57,399:    INFO:      ard_logging: MainProcess 3343: Logging initialized.
2022-06-20 13:45:57,400:    INFO:         __main__: MainProcess 3343: Hello from notebook


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [18]:
import ard_fit_pipelines as fp
reload(fp.core); reload(fp.transforms); reload(fp)
import ard_fit_pipelines.core as fpc
import ard_fit_pipelines.transforms as fpt

logging.getLogger('ard_fit_pipelines').setLevel(logging.DEBUG)
#reload(fpc)

In [16]:
t = fpt.DeMean(cols=fpc.hp('response_cols'))
tfit = t.fit(df, bindings={'response_cols':['price']})
tfit.apply(df).head()

fpt.CopyColumns(cols=['price'], dest_cols=['price_copy_1', 'price_copy_2']).apply(df).head()

2022-06-20 12:37:28,789:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Fitting DeMean on 53940 rows: DeMean(cols=hp(name='response_cols'))
2022-06-20 12:37:28,793:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Applying DeMean.FitDeMean to 53940 rows: FitDeMean(cols=['price'], <53940 rows of fitting data>)
2022-06-20 12:37:28,797:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Fitting CopyColumns on 0 rows: CopyColumns(cols=['price'], dest_cols=['price_copy_1', 'price_copy_2'])
2022-06-20 12:37:28,797:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Applying CopyColumns.FitCopyColumns to 53940 rows: FitCopyColumns(cols=['price'], dest_cols=['price_copy_1', 'price_copy_2'], <0 rows of fitting data>)


cols: Bound hp(name='response_cols') -> ['price']
cols: Bound ['price'] -> ['price']
dest_cols: Bound ['price_copy_1', 'price_copy_2'] -> ['price_copy_1', 'price_copy_2']


,carat,cut,color,clarity,depth,table,price,x,y,z,price_copy_1,price_copy_2
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,326,326
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,326,326
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,327,327
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,334,334
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,335,335


In [30]:
cols = ['price', 'x', 'y', 'z']
t = fpt.DeMean(cols)
result = t.fit(df).apply(df)
(result[cols].mean().abs() < 1e-10).all()

2022-06-20 13:57:00,151:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Fitting DeMean on 53940 rows: DeMean(cols=['price', 'x', 'y', 'z'])
2022-06-20 13:57:00,156:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Applying DeMean.FitDeMean to 53940 rows: FitDeMean(cols=['price', 'x', 'y', 'z'], <53940 rows of fitting data>)


cols: Bound ['price', 'x', 'y', 'z'] -> ['price', 'x', 'y', 'z']


True

In [19]:
class DeMean(fpc.ColumnsTransform):
    "Docstr for MyDeMean"
    
    def _fit(self, X_fit):
        return X_fit[self.cols].mean()
    
    def _apply(self, X_apply, state):
        means = state
        return X_apply.assign(**{
            c: X_apply[c] - means[c]
            for c in self.cols
        })
        

#t = DeMean(cols=['price'])
t = DeMean(cols=fpc.hp('response_cols'))
tfit = t.fit(df, bindings={'response_cols':['price']})
tfit.apply(df).head()

2022-06-20 13:46:03,738:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Fitting DeMean on 53940 rows: DeMean(cols=hp(name='response_cols'))
2022-06-20 13:46:03,748:   DEBUG: ard_fit_pipelines.core: MainProcess 3343: Applying DeMean.FitDeMean to 53940 rows: FitDeMean(cols=['price'], <53940 rows of fitting data>)


cols: Bound hp(name='response_cols') -> ['price']


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,-3606.799722,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,-3606.799722,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,-3605.799722,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,-3598.799722,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,-3597.799722,4.34,4.35,2.75


In [24]:
repr(t)

"DeMean(cols=hp(name='response_cols'))"

In [25]:
repr(tfit)

"FitDeMean(cols=['price'], <53940 rows of fitting data>)"

In [23]:
isinstance(DeMean.FitDeMean, type) 

True

In [177]:
t = fpc.Identity()
t.apply(df).head()

2022-06-20 12:10:52,908:   DEBUG: ard_fit_pipelines.core: MainProcess 2337: Fitting Identity on 0 rows: Identity()
2022-06-20 12:10:52,909:   DEBUG: ard_fit_pipelines.core: MainProcess 2337: Applying Identity.FitIdentity to 53940 rows: FitIdentity(<0 rows of fitting data>)


,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [137]:
help(DeMean.FitDeMean)

Help on class FitDeMean in module ard_fit_pipelines.core:

class FitDeMean(FitTransform)
 |  FitDeMean(transform: 'DeMean', X_fit: 'pd.DataFrame', bindings=None)
 |  
 |  The result of fitting a DeMean Transform. Blah blah.
 |  
 |  Method resolution order:
 |      FitDeMean
 |      FitTransform
 |      abc.ABC
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from FitTransform:
 |  
 |  __init__(self, transform: 'DeMean', X_fit: 'pd.DataFrame', bindings=None)
 |      Docstr for FitTransform.__init__
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  apply(self, X_apply: 'pd.DataFrame') -> 'pd.DataFrame'
 |      Docstr for FitTransform.apply
 |  
 |  state(self)
 |  
 |  ----------------------------------------------------------------------
 |  Class methods inherited from FitTransform:
 |  
 |  __init_subclass_

In [10]:
DeMean.fit?

Signature:
DeMean.fit(
    self,
    X_fit: 'pd.DataFrame',
    bindings: 'Optional[dict[str, object]]' = None,
) -> 'DeMean.FitDeMean'
Docstring: <no docstring>
File:      ~/projects/ardua/codebases/local/ard-fit-pipelines/src/ard_fit_pipelines/core.py
Type:      function


In [13]:
tfit